# Tarea 1: Detección de Entidades Nombradas
#### Utilización de un etiquetador de entidades nombradas, en español, aplicado al problema planteado en el artículo [Introduction to the CoNLL-2002 Shared Task: Language-Independent Named Entity Recognition](https://www.aclweb.org/anthology/W02-2024).

### Objetivos:
- Utilizar el NER preentrenado de spaCy sobre el corpus CoNLL-2002 ([esp.testb](https://www.clips.uantwerpen.be/conll2002/ner/data/esp.testb))
- Analizar los resultados contra las etiquetas del conjunto de test.
- Calcular la precisión, cobertura y medida-F utilizando conlleval.py ([conlleval.py](http://localhost:8888/lab/tree/practicas/T1/T1-NER-CoNLL-2002.ipynb))

Documentación a entregar:
- Documento PDF (en ningun caso se entregarán archivos jupyter) con la siguiente información:
    - Descripción de la tarea de conll2002 (comentar los principales puntos y resultados del artículo) y los datos de evaluación.
    - Descripción del código desarrollado, herramientas utilizadas, etc.
    - Textos de prueba utilizados.
    - Resultados de evaluación del etiquetado.
    - Análisis de los errores de etiquetado y sus causas. Se valorará la introducción de mejoras/modificaciones en el etiquetado que mejoren los resultados.
- Fichero comprimido con el co ́digo fuente y los ficheros generados.

In [1]:
! wget https://www.clips.uantwerpen.be/conll2002/ner/data/esp.testb -P ./src

conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 410611 (401K)
Grabando a: “./src/esp.testb.1”

esp.testb.1         100%[===================>] 400,99K  1,86MB/s    en 0,2s    

2020-12-15 03:15:38 (1,86 MB/s) - “./src/esp.testb.1” guardado [410611/410611]



In [2]:
esp_test_raw_data = open('./src/esp.testb', 'r')

In [3]:
import re
# Text:
text = ''
# Original Solution
anotated_reference = {}
for i, line in enumerate(esp_test_raw_data):
    anotated_word = re.split('\s', line)
    word = anotated_word[0]
    anotation = anotated_word[1]
    # Save original Solution
    anotated_reference[i] = (word, anotation)
    # Prepare text input for spaCy's NER model.
    text += word + ' '
    
# Close file after extracting info.
esp_test_raw_data.close()

In [4]:
import spacy
# Download Model:
import spacy.cli
spacy.cli.download("es")
# Load Model
es_model = spacy.load('es')

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/Users/adzarei/Documents/UNED/ING_CIENC_DATOS/projects/MDT/conda-env/lib/python3.8/site-packages/es_core_news_sm
-->
/Users/adzarei/Documents/UNED/ING_CIENC_DATOS/projects/MDT/conda-env/lib/python3.8/site-packages/spacy/data/es
You can now load the model via spacy.load('es')


In [5]:
anotated_doc_es = es_model(text)

In [6]:
anotated_output = {}
for i, token in enumerate(anotated_doc_es):
    anotated_output[i] = (token.text, token.ent_iob_ + "-" + token.ent_type_ if token.ent_iob_ != 'O' else token.ent_iob_)

#### Generamos el fichero de salida a partir de `anotated_reference`y `anotated_output` 

In [7]:
import datetime

log_file = open("./src/log_testb.log", "a")
log_file.write('Starting Execution: {}'.format(datetime.datetime.now()))

out_file = open("./src/esp_out.testb", "w")

j = 0
max_errors = 10
for idx in anotated_reference:
    word = anotated_reference[idx][0]
    ref  = anotated_reference[idx][1]
    
    # If there is a miss match, we try with the next elements of anotated_output.
    while(word.strip() != anotated_output[j][0].strip()):
        log_file.write('Word missmatch: ref:{} asig: {}\n'.format(word, anotated_output[j][0]))
        j += 1
        # If there are more 
        if j - idx > max_errors:
            log_file.write('Max Errors reached ({})\n'.format(max_errors))
            j = idx
            break
    
    asig = '' if word.strip() == '' else anotated_output[j][1]
    out_file.write('{} {} {}\n'.format(word, ref, asig))
    j += 1
    
out_file.close()
log_file.close()

In [8]:
! python ./src/conlleval.py < ./src/esp_out.testb

processed 53049 tokens with 3559 phrases; found: 3802 phrases; correct: 316.
accuracy:   8.90%; (non-O)
accuracy:  80.70%; precision:   8.31%; recall:   8.88%; FB1:   8.59
              LOC: precision:   7.52%; recall:  10.06%; FB1:   8.60  1450
             MISC: precision:   2.26%; recall:   4.41%; FB1:   2.99  663
              ORG: precision:  12.31%; recall:   7.50%; FB1:   9.32  853
              PER: precision:  10.41%; recall:  11.84%; FB1:  11.08  836
